# BYU Graduate Program Analysis

BYU posts data about each of it's graduate programs online. Which programs are the hardest to get into? Which programs have the highest test scores or the lowest? Which programs are the longest? Let's scrape the data and then do some analysis. 

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

### Scraping the data
For this project, we will use selenium to open a web browser and follow a couple simple input commands. We then will parse the HTML and retrieve the data.

In [32]:
#Urls
gradStudies = 'https://gradstudies.byu.edu'
programsUrl = 'https://gradstudies.byu.edu/programs'
#Initialize Web Driver
browser = webdriver.Chrome()

In [12]:
#Navigate to the list of all programs
browser.get(programsUrl)

#Process HTML
allProgHTML = browser.page_source
soup = BeautifulSoup(allProgHTML,'html.parser')

progPages = {
    'Name' : [],
    'Link' : []
}

programDiv = soup.find('div',attrs={'class' : 'catalog-list-item'})
while programDiv:
    #Find a program names and respective url
    link = programDiv.find('a')
    if link:
        progPages['Link'].append(gradStudies + str(link.get('href')))
        progPages['Name'].append(link.text.strip())
    
    #Iterate (Go to the next program)
    programDiv = programDiv.find_next_sibling()

In [33]:

#Dictionary to store the data
progData = {'Program':[]}
#Make the old dictionary into tuples
LinkName = zip(progPages['Link'],progPages['Name'])

#Visit each program page and get the data
for linkname in LinkName:
    browser.get(linkname[0])
    progData['Program'].append(linkname[1])
    
    #Parse HTML
    soup = BeautifulSoup(browser.page_source,'html.parser')
    table = soup.find('table',attrs={'class':'table table-hover'})
    while table:
        row = table.find('tr')
        while row:
            #Find the data
            column1 = row.find('td')
            if column1:
                factorName = column1.text.strip()
                factorValue = column1.find_next('td').text.strip()
            
                #Check if factor is already in dictionary
                try:
                    progData[factorName].append(factorValue)
                #If not, fill the rows above with Nones
                except KeyError:
                    numRows = len(progData['Program']) - 1
                    progData[factorName] = [None]*numRows + [factorValue]
            
            #Iterate
            row = row.find_next_sibling()
        table = table.find_next_sibling()
        
    #After completeing each program page, fill in any unused factors with Nones
    for key in progData.keys():
        if len(progData[key]) < len(progData['Program']):
            progData[key].append(None)
        
    

### Data Cleaning
What we just scraped is a dataset of the admission and program statistics of BYU graduate programs.

In [170]:
import pandas as pd
import numpy as np

In [171]:
#Turn the data into a dataframe and save it
gradStats = pd.DataFrame(progData)
gradStats.fillna(value=np.nan, inplace=True)
gradStats.to_csv('gradStatsBYU.csv')

In [172]:
gradStats.describe()

,Admitted Per Year,Applied Per Year,Average GPA,Average Years to Degree,BYU Undergraduate,Female,GMAT Analytical Percentile,GMAT Analytical Score,GMAT Composite Score,GMAT Integrated Percentile,...,GRE Verbal Percentile,GRE Verbal Score,Graduated Per Year,International,LDS,LSAT Composite Score,Non-White,Percent Admitted,Program,Total Students
count,86,86,86,88,85,85,6,6,6,6,...,70,70,88,85,85,1,85,88,89,85
unique,69,77,40,37,41,32,6,4,6,6,...,61,50,65,15,50,1,21,80,89,49
top,4.0,8.4,3.67,2.0,4,5,39.3,4.0,615.4,57.9,...,78.0,155.8,2.0,0,20,160.8,1,40.00,Law (JD),14
freq,4,2,6,5,7,8,1,2,1,1,...,3,3,4,22,5,1,13,3,1,5


A problem in being able to compare programs is that different subjects require different tests. The main exams are GRE, GMAT and LSAT. We will divide up the database accordingly.

In [173]:
programs = gradStats.Program

#Mask all programs with no GMAT data
mask = ~gradStats['GMAT Analytical Score'].isnull()
GMATProg = gradStats.loc[mask,:]

#Mask programs with no LSAT data
mask = ~gradStats['LSAT Composite Score'].isnull()
LSATProg = gradStats.loc[mask,:]

#Mask programs with no GRE data
mask = ~gradStats['GRE Analytical Score'].isnull()
GREProg = gradStats.loc[mask,:]

In [174]:
GREProg.describe()

,Admitted Per Year,Applied Per Year,Average GPA,Average Years to Degree,BYU Undergraduate,Female,GMAT Analytical Percentile,GMAT Analytical Score,GMAT Composite Score,GMAT Integrated Percentile,...,GRE Verbal Percentile,GRE Verbal Score,Graduated Per Year,International,LDS,LSAT Composite Score,Non-White,Percent Admitted,Program,Total Students
count,70,70,70,70,69,69,2,2,2,2,...,70,70,70,69,69,0.0,69,70,70,69
unique,55,64,36,34,31,28,2,2,2,2,...,61,50,54,13,40,0.0,18,66,70,40
top,2.0,16.0,3.63,2.2,13,6,49.3,4.3,540.5,57.9,...,78.0,155.8,2.0,0,20,NaN,1,40.00,Microbiology and Molecular Biology (PhD),14
freq,3,2,5,5,6,7,1,1,1,1,...,3,3,3,20,5,NaN,12,3,1,5


Now we will remove all columns that do not apply to the programs in each set, except for in the GRE programs, where one program does not include the typical data. (This can be seen from the summary above.) We will find this program and remove it before removing the other empty columns.

In [175]:
#Remove spurious columns
count = GMATProg.describe().loc['count']
GMATProg = GMATProg[GMATProg.columns[count == count['Admitted Per Year']]]

#Remove spurious columns
count =LSATProg.describe().loc['count']
LSATProg = LSATProg[LSATProg.columns[count == count['Admitted Per Year']]]

#Find offender
offender = GREProg.Program[GREProg.Female.isnull()]
#Remove offender
GREProg.drop(offender.index,axis=0,inplace=True)
#Remove spurious columns
count = GREProg.describe().loc['count']
GREProg = GREProg[GREProg.columns[count == count['Admitted Per Year']]]


/Library/Python/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Since all of the data is numeric except for the program name, let's assign float values to the data.

In [176]:
for df in [GREProg,GMATProg,LSATProg]:
    for col in df.columns:
        if col != 'Program':
            df[col] = df[col].astype(float)

In [177]:
GMATProg.describe()

,Admitted Per Year,Applied Per Year,Average GPA,Average Years to Degree,BYU Undergraduate,Female,GMAT Analytical Percentile,GMAT Analytical Score,GMAT Composite Score,GMAT Integrated Percentile,...,GMAT Quantitative Percentile,GMAT Quantitative Score,GMAT Verbal Percentile,GMAT Verbal Score,Graduated Per Year,International,LDS,Non-White,Percent Admitted,Total Students
count,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,...,6.000000,6.00000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
mean,90.833333,138.133333,3.603333,1.266667,84.833333,31.166667,49.650000,4.350000,638.766667,56.466667,...,53.916667,41.45000,74.350000,36.000000,78.533333,15.333333,123.166667,26.500000,72.490000,128.833333
std,54.982094,109.637269,0.113783,0.628225,41.199110,21.949184,8.230128,0.403733,58.208476,9.102234,...,13.615200,5.32156,7.997187,2.761159,40.639423,21.030137,86.142711,27.631504,13.389497,92.527654
min,51.600000,57.400000,3.400000,0.700000,57.000000,7.000000,39.300000,4.000000,540.500000,48.500000,...,35.200000,33.40000,60.000000,30.700000,46.000000,4.000000,60.000000,7.000000,56.370000,60.000000
25%,62.850000,77.450000,3.567500,0.700000,62.250000,15.500000,45.675000,4.075000,617.375000,50.425000,...,46.000000,38.77500,72.600000,35.925000,53.400000,5.250000,74.500000,10.250000,65.740000,76.250000
50%,68.300000,99.900000,3.645000,1.200000,68.500000,27.000000,48.050000,4.300000,647.650000,54.600000,...,54.150000,41.65000,76.650000,36.650000,62.600000,7.500000,94.000000,15.500000,67.850000,96.000000
75%,89.650000,136.450000,3.670000,1.775000,83.750000,43.750000,52.750000,4.375000,685.500000,57.650000,...,61.475000,45.20000,77.925000,37.675000,87.850000,9.750000,121.750000,28.250000,82.912500,130.750000
max,199.000000,353.000000,3.710000,2.000000,166.000000,65.000000,63.300000,5.100000,691.400000,73.400000,...,72.800000,47.80000,83.200000,38.300000,153.400000,58.000000,292.000000,80.000000,89.900000,309.000000


In [178]:
LSATProg.describe()

,Admitted Per Year,Applied Per Year,Average GPA,Average Years to Degree,BYU Undergraduate,Female,Graduated Per Year,International,LDS,LSAT Composite Score,Non-White,Percent Admitted,Total Students
count,1.0,1.0,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mean,204.4,544.8,3.64,2.7,221.0,140.0,138.4,12.0,375.0,160.8,65.0,36.4,386.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,204.4,544.8,3.64,2.7,221.0,140.0,138.4,12.0,375.0,160.8,65.0,36.4,386.0
25%,204.4,544.8,3.64,2.7,221.0,140.0,138.4,12.0,375.0,160.8,65.0,36.4,386.0
50%,204.4,544.8,3.64,2.7,221.0,140.0,138.4,12.0,375.0,160.8,65.0,36.4,386.0
75%,204.4,544.8,3.64,2.7,221.0,140.0,138.4,12.0,375.0,160.8,65.0,36.4,386.0
max,204.4,544.8,3.64,2.7,221.0,140.0,138.4,12.0,375.0,160.8,65.0,36.4,386.0


In [179]:
GREProg.describe()

,Admitted Per Year,Applied Per Year,Average GPA,Average Years to Degree,BYU Undergraduate,Female,GRE Analytical Percentile,GRE Analytical Score,GRE Composite Score,GRE Quantitative Percentile,GRE Quantitative Score,GRE Verbal Percentile,GRE Verbal Score,Graduated Per Year,International,LDS,Non-White,Percent Admitted,Total Students
count,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000
mean,14.213043,25.613043,3.609710,3.265217,19.057971,11.347826,59.842029,4.110145,312.843478,61.053623,155.586957,72.053623,157.253623,10.584058,4.130435,28.260870,6.507246,57.330870,30.376812
std,25.052169,43.943088,0.108098,1.390707,24.007893,11.309582,10.764216,0.299089,6.463187,16.249177,5.237560,9.051691,2.742561,19.181292,8.479930,37.711229,10.986118,15.847331,39.617324
min,2.000000,3.400000,3.360000,1.300000,1.000000,0.000000,33.400000,3.400000,296.800000,33.500000,138.400000,50.000000,151.200000,1.500000,0.000000,2.000000,0.000000,19.810000,2.000000
25%,4.800000,8.200000,3.550000,2.200000,8.000000,4.000000,54.300000,4.000000,309.200000,48.100000,152.000000,67.000000,155.600000,3.400000,0.000000,12.000000,2.000000,47.140000,13.000000
50%,9.000000,14.600000,3.620000,2.800000,13.000000,7.000000,61.800000,4.100000,311.700000,60.100000,154.800000,71.700000,157.000000,6.000000,2.000000,19.000000,4.000000,58.000000,21.000000
75%,14.400000,27.000000,3.690000,4.000000,19.000000,15.000000,66.000000,4.300000,318.400000,76.300000,159.400000,78.200000,159.000000,11.400000,4.000000,31.000000,8.000000,68.660000,36.000000
max,199.000000,353.000000,3.860000,6.800000,166.000000,65.000000,83.000000,4.800000,327.400000,91.000000,165.300000,89.600000,163.800000,153.400000,58.000000,292.000000,80.000000,85.710000,309.000000


Finally, we will set program name as the index of each dataframe. Since there is only one law program, we will get rid of it.

In [180]:
LSATProg = None

GREProg.index = GREProg.Program
GREProg.drop("Program",axis=1)

GMATProg.index = GMATProg.Program
GMATProg.drop("Program",axis=1)

,Admitted Per Year,Applied Per Year,Average GPA,Average Years to Degree,BYU Undergraduate,Female,GMAT Analytical Percentile,GMAT Analytical Score,GMAT Composite Score,GMAT Integrated Percentile,...,GMAT Quantitative Percentile,GMAT Quantitative Score,GMAT Verbal Percentile,GMAT Verbal Score,Graduated Per Year,International,LDS,Non-White,Percent Admitted,Total Students
Program,,,,,,,,,,,,,,,,,,,,,
Accountancy - Professional (MAcc),95.2,146.2,3.71,0.7,88.0,13.0,39.3,4.0,691.4,56.9,...,61.7,45.6,71.3,38.3,96.0,5.0,85.0,10.0,65.12,89.0
Accountancy - Tax (MAcc),63.6,72.4,3.67,0.7,71.0,31.0,63.3,5.1,690.0,52.3,...,72.8,47.8,76.8,37.0,63.4,4.0,71.0,7.0,87.85,72.0
Business Administration (MBA),199.0,353.0,3.55,1.7,166.0,65.0,53.9,4.4,672.0,57.9,...,60.8,44.0,83.2,37.9,153.4,58.0,292.0,80.0,56.37,309.0
Business Administration - Executive Program (MBA),73.0,107.2,3.40,2.0,66.0,23.0,46.8,4.0,615.4,48.5,...,45.5,38.6,76.5,35.8,61.8,9.0,128.0,20.0,68.10,140.0
Information Systems Management (MISM),51.6,57.4,3.67,0.7,57.0,7.0,45.3,4.3,623.3,73.4,...,47.5,39.3,78.3,36.3,50.6,6.0,60.0,11.0,89.90,60.0
Public Administration (MPA),62.6,92.6,3.62,1.8,61.0,48.0,49.3,4.3,540.5,49.8,...,35.2,33.4,60.0,30.7,46.0,10.0,103.0,31.0,67.60,103.0
